# Multi-agent Collaboration for Financial Analysis 📊 

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()

#os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [9]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [10]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [11]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [12]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [13]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [14]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

- Example : Moderate-Risk Day Trading Strategy with News Sensitivity

In [15]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [16]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the team is effectively monitoring and analyzing market data for AAPL to identify trends and predict market movements. I should delegate tasks and ask questions to the right team members to get the best insights and alerts about significant market opportunities or threats.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning techniques", "context": "The goal is to identify trends and predict market movements for AAPL."}

> Entering new CrewAgentExecutor chain...
Thought: To monitor and analyze market data for AAPL, I need to find a reliable source that provi

- Display the final result as Markdown.

In [17]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for AAPL Trading Strategies:

Based on the analysis of the proposed trading strategies for AAPL and the potential risks associated with each approach, it is essential for traders to consider the following factors:

1. Volatility of AAPL Stock: AAPL has shown significant fluctuations in yearly returns, with a -26% decline in 2022 and a +49% increase in 2023. This volatility can pose risks for traders using strategies like direct share purchases and ETF trading.

2. Market Uncertainties: Recent concerns over weaker iPhone demand and potential antitrust litigation against Apple could introduce further instability in AAPL's stock price. This uncertainty can impact trading strategies like options trading and Contracts for Difference (CFDs).

3. Risk Management Tools: Utilizing risk management tools such as 'Close at Profit', 'Close at Loss', and 'Guaranteed Stop' can help traders protect their investments and navigate market uncertainties. However, setting these parameters thoughtfully based on current market analysis is crucial.

Recommendations for Mitigating Risks:

1. Diversification: Traders should consider diversifying their trading strategies to minimize risks associated with individual stocks or sectors. Combining multiple approaches can help spread risk and increase the chances of success.

2. Informed Decision-Making: By combining fundamental and technical analysis, traders can make informed decisions based on AAPL's financial performance, product launches, earnings reports, and broader market trends. This knowledge can guide strategic adjustments to trading plans.

3. Adaptability: Traders should be prepared to adapt their strategies to changing market conditions and news events. Being flexible and responsive to emerging risks can help mitigate potential losses and optimize trading opportunities.

In conclusion, while the proposed trading strategies for AAPL offer opportunities for potential returns, traders must carefully evaluate and manage the associated risks. By considering the factors outlined above and implementing sound risk management practices, traders can increase their chances of success in trading AAPL effectively.

**Explanation output** TO BE DRAFTED 

### Extra investigation

To create another reliable scenario in your CrewAI-based multi-agent financial analysis, you can modify the parameters based on your desired trading approach and risk profile. Here’s how you can tweak them:

Key Adjustments for Different Scenarios
Stock Selection (stock_selection)

Change the stock based on market conditions or industry preferences.
Example: 'TSLA' (Tesla) for a high-volatility scenario or 'JNJ' (Johnson & Johnson) for a more stable investment.
Initial Capital (initial_capital)

Adjust the capital to simulate different investor types.
Example: Lower value (e.g., 5000) for small retail investors or higher value (e.g., 500000) for institutional-style trading.
Risk Tolerance (risk_tolerance)

'Low' → More conservative, lower-risk stocks & safe trading strategies.
'Medium' → Balanced approach (mix of growth & stability).
'High' → Aggressive, higher volatility, higher reward strategies.
Trading Strategy (trading_strategy_preference)

'Day Trading' → High-frequency, short-term trades.
'Swing Trading' → Medium-term trades, holding positions for days/weeks.
'Long-Term Investing' → Investing in stocks for months/years.
News Impact (news_impact_consideration)

True → Incorporates news sentiment analysis into decision-making.
False → Ignores external news, relies solely on technical analysis.

- Example: More Conservative Long-Term Scenario

In [18]:
financial_trading_inputs = {
    'stock_selection': 'JNJ',  # Stable stock
    'initial_capital': '200000',  # Higher capital for long-term investment
    'risk_tolerance': 'Low',  # Conservative approach
    'trading_strategy_preference': 'Long-Term Investing',  # Holding stocks for long periods
    'news_impact_consideration': False  # Ignores short-term news fluctuations
}


In [19]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (JNJ). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for the selected stock (JNJ) to the Data Analyst. They have the expertise in statistical modeling and machine learning to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for JNJ using statistical modeling and machine learning", "context": "The goal is to identify trends and predict market movements for JNJ"}

> Entering new CrewAgentExecutor chain...
Thought: To monitor and analyze market data for JNJ (Johnson & Johnson), I need up-to-date financial market data. Websites like Bloomberg, Reuters, or financia

In [20]:
# Display the final result as Markdown.
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report:

Title: Comprehensive Risk Analysis and Mitigation Recommendations for LSTM-Based Trading Strategies on Johnson & Johnson (JNJ)

Executive Summary:
This report provides a detailed analysis of potential risks associated with utilizing LSTM models for trading Johnson & Johnson (JNJ) stock. The proposed trading strategies include risk management through stop-loss and take-profit orders, rebalancing strategy, performance monitoring, and feedback loop mechanisms. Mitigation recommendations are suggested to enhance the effectiveness of the strategies.

Risk Analysis:
1. Market Dynamics:
   - JNJ stock exhibits stability with a beta of 0.51, indicating lower volatility.
   - Analyst ratings range from "Buy" to "Hold", signaling a cautious optimism in the market.

2. Risks of LSTM Models:
   - LSTM models are effective in predicting stock prices but are susceptible to data quality issues.
   - Accuracy rates in other markets show a moderate success rate of 53.8%.

Mitigation Recommendations:
1. Data Quality Assurance:
   - Ensure high-quality data inputs to minimize overfitting and enhance model performance.
   
2. Continuous Evaluation:
   - Regularly assess the LSTM model's accuracy against actual market outcomes for adjustments.
   
3. Diversification:
   - Use LSTM predictions as one of several tools for decision-making to reduce reliance on a single model.
   
4. Risk Management Strategies:
   - Implement stop-loss and take-profit orders to manage potential losses and secure profits.
   
5. Rebalancing:
   - Adjust the portfolio quarterly based on model predictions to maintain a balanced risk profile.
   
6. Feedback Loop:
   - Incorporate trading outcomes back into the model for refinement to adapt to market shifts.

Conclusion:
The comprehensive risk analysis highlights the importance of mitigating potential risks associated with LSTM-based trading strategies for JNJ. By implementing the recommended mitigation strategies, the project can enhance its risk management practices and optimize trading performance in alignment with the firm's risk tolerance and market conditions.

**Explanation output** TO BE DRAFTED 

- Example: High-Risk, High-Reward Day Trading

In [21]:
financial_trading_inputs = {
    'stock_selection': 'TSLA',  # Volatile stock
    'initial_capital': '50000',  # Mid-level capital for active trading
    'risk_tolerance': 'High',  # Aggressive trading style
    'trading_strategy_preference': 'Day Trading',  # Frequent trades
    'news_impact_consideration': True  # Market-sensitive trades
}


In [22]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (TSLA). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for TSLA to the Data Analyst, as they have the expertise in statistical modeling and machine learning.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "Data Analyst",
    "task": "Continuously monitor and analyze market data for TSLA",
    "context": "The task involves using statistical modeling and machine learning to identify trends and predict market movements for TSLA."
}


> Entering new CrewAgentExecutor chain...
Thought: To continuously monitor and analyze market data for TSLA, I need to find a reliable source that provides real-time data on TSLA's stock performance.

Action: Search the internet
Action Input

In [23]:
# Display the final result as Markdown.
from IPython.display import Markdown
Markdown(result)

The proposed trading strategies involving TSLA, leveraging Single Stock Daily Leveraged & Inverse ETFs and options trading, present several risk factors that need careful management:

1. **Leveraged and Inverse ETFs:**
   - **Compounding Risk:** Due to daily resets, leveraged and inverse ETFs can lead to losses that are magnified over time, especially in volatile markets.
   - **Market Risk:** These ETFs are susceptible to rapid price changes. The inverse ETFs, in particular, pose a high risk if the market moves contrary to the expected direction.
   - **Liquidity and Volume Issues:** These ETFs may not always have sufficient trading volume, leading to potential difficulties in entering or exiting positions without affecting the price.

   **Mitigation Strategies:**
   - Limit the allocation to leveraged and inverse ETFs within the portfolio to manage exposure.
   - Employ stop-loss orders to help limit potential losses.
   - Continuously monitor the ETFs for performance divergences from expected results and adjust strategies as necessary.

2. **Options Trading with TSLA:**
   - **Market Movements:** Significant losses can occur if the market moves against the position, especially with strategies like selling naked options.
   - **Volatility:** TSLA options can exhibit high volatility, which can both create opportunities and significantly increase risk.
   - **Complexity of Products:** Options strategies can be complex, and mismanagement or misunderstanding of these products can lead to significant losses.

   **Mitigation Strategies:**
   - Utilize options strategies that limit risk, such as spreads, rather than strategies with unlimited risk (e.g., naked calls).
   - Implement rigorous risk management practices, including regular monitoring of the options' market movements and adjustments to strategies as market conditions change.
   - Educate traders on the complexities and risks of each options strategy being used.

3. **General Risk Management:**
   - **Stay Informed:** Keep updated with real-time market data, particularly following recent technical issues with Nasdaq, which could affect trading.
   - **Diversification:** Avoid concentration in TSLA alone; diversify trades among different securities and asset classes.
   - **Continuous Education:** Regularly update training for traders on the latest market conditions, trading techniques, and compliance requirements.

By adhering to these strategies, the firm can better manage the risks associated with trading TSLA using leveraged and inverse ETFs and options, aligning activities with the firm’s overall risk tolerance and investment goals.

All these examples use a sequential workflow. Sequential Execution (Default)
✔ How It Works:

Each agent waits for the previous agent to complete its task before proceeding.
Ensures step-by-step validation and structured decision-making.
Good for controlled environments, but may be slower in fast-moving markets.

✔ Example Flow:

[Data Analyst] → [Strategy Agent] → [Risk Manager] → [Execution Agent]


✔ Best For:

Low-frequency trading (Swing Trading, Long-Term Investing).
Ensuring every step is fully validated before trade execution.

Since you’re using CrewAI for financial trading, I recommend using a hybrid approach to:
✔ Speed up data processing & risk assessment (parallel).
✔ Keep strategy formulation & execution structured (sequential).

Hybrid Execution (Best of Both Worlds)
✔ How It Works:

Some agents run in parallel, others remain sequential where needed.
Example:
Data Analyst & Risk Manager work in parallel
Strategy Agent waits for both before deciding
Execution Agent waits for final approval
✔ Optimized Flow:

[Data Analyst] → \
                   → [Strategy Agent] → [Execution Agent]
[Risk Manager] → /


✔ Best For:

Day trading systems with risk-sensitive adjustments.
Balancing speed with decision accuracy.
Adaptive trading bots that need real-time risk monitoring.

In [25]:
from IPython.core.display import display, HTML

css_style = """
<style>
.flow-container {
    font-family: Arial, sans-serif;
    font-size: 14px;
    background-color: #f8f9fa;
    padding: 15px;
    border-radius: 10px;
    border: 1px solid #ddd;
    width: fit-content;
}
.flow-step {
    padding: 8px 15px;
    margin: 5px;
    background-color: #007bff;
    color: white;
    border-radius: 5px;
    display: inline-block;
}
.arrow {
    font-size: 16px;
    font-weight: bold;
    color: #333;
}
</style>
"""

flow_html = """
<div class='flow-container'>
    <span class='flow-step'>Data Analyst</span>
    <span class='arrow'>→</span>
    <span class='flow-step'>Strategy Agent</span>
    <span class='arrow'>→</span>
    <span class='flow-step'>Risk Manager</span>
    <span class='arrow'>→</span>
    <span class='flow-step'>Execution Agent</span>
</div>
"""

display(HTML(css_style + flow_html))


In [26]:
from IPython.core.display import display, HTML

css_style = """
<style>
.flow-container {
    font-family: Arial, sans-serif;
    font-size: 14px;
    background-color: #f8f9fa;
    padding: 15px;
    border-radius: 10px;
    border: 1px solid #ddd;
    width: fit-content;
}
.flow-row {
    display: flex;
    align-items: center;
    justify-content: center;
    margin-bottom: 10px;
}
.flow-step {
    padding: 8px 15px;
    margin: 5px;
    background-color: #007bff;
    color: white;
    border-radius: 5px;
    display: inline-block;
}
.arrow {
    font-size: 16px;
    font-weight: bold;
    color: #333;
    margin: 0 8px;
}
.parallel-container {
    display: flex;
    justify-content: center;
}
</style>
"""

flow_html = """
<div class='flow-container'>

    <!-- Parallel Execution Row -->
    <div class='parallel-container'>
        <div class='flow-step'>Data Analyst</div>
        <span class='arrow'>⇄</span>
        <div class='flow-step'>Risk Manager</div>
    </div>

    <!-- Sequential Execution -->
    <div class='flow-row'>
        <span class='arrow'>↓</span>
    </div>

    <div class='flow-row'>
        <div class='flow-step'>Strategy Agent</div>
    </div>

    <div class='flow-row'>
        <span class='arrow'>↓</span>
    </div>

    <div class='flow-row'>
        <div class='flow-step'>Execution Agent</div>
    </div>

</div>
"""

display(HTML(css_style + flow_html))
